In [1]:
#ensemble learning methods on 4 datasets with 4 train test split combinations and diff set of hyperparameter values

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

#models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC

In [79]:
df=pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [81]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [82]:
def preprocessing(df):
    title_dictionary = {'Capt': 'Dr/Clergy/Mil','Col': 'Dr/Clergy/Mil','Major': 'Dr/Clergy/Mil','Jonkheer': 'Honorific','Don': 'Honorific',
                        'Dona': 'Honorific','Sir': 'Honorific','Dr': 'Dr/Clergy/Mil','Rev': 'Dr/Clergy/Mil','the Countess': 'Honorific',
                        'Mme': 'Mrs','Mlle': 'Miss','Ms': 'Mrs','Mr': 'Mr','Mrs': 'Mrs','Miss': 'Miss','Master': 'Master','Lady': 'Honorific'}
    df['Title']=df['Name'].map(lambda name: name.split(',')[1].split('.')[0].strip())
    df['Title']=df['Title'].map(title_dictionary)
    values=list(title_dictionary.values())
    title_dict={values[items]:items+1 for items in range(len(values))}
    df.Title=df.Title.map(title_dict)
    
    df['Embarked'].fillna(df.Embarked.mode()[0], inplace=True)
    df['Embarked']=df['Embarked'].map({'S':0,'C':1,'Q':2})
    
    df['Sex']=df['Sex'].map({'male':0,'female':1})
    
    df['Age'].fillna(df.Age.mean(), inplace=True)
    
    df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [83]:
preprocessing(df)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,0,14
1,1,1,1,38.0,1,0,71.2833,1,15
2,1,3,1,26.0,0,0,7.9250,0,16
3,1,1,1,35.0,1,0,53.1000,0,15
4,0,3,0,35.0,0,0,8.0500,0,14


In [84]:
df.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,0.361392,14.611672
std,0.486592,0.836071,0.477990,13.002015,1.102743,0.806057,49.693429,0.635673,1.262973
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,9.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000,14.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,14.454200,0.000000,14.000000
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,1.000000,16.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000,18.000000


In [106]:
x=df.iloc[:,1:]
y=df.iloc[:,0]

In [107]:
sc=StandardScaler()
x=sc.fit_transform(x)

In [157]:
def fit_model(x,y):
    split=0.1
    for i in range(4):
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=split,random_state=42)
        dt = DecisionTreeClassifier()
        knn = KNeighborsClassifier()
        svc = SVC()
        split+=0.1
        clf_models=[dt,knn,svc]
        kfold = model_selection.KFold(n_splits = 11,shuffle=True, random_state = 42)
        print('Test Size: ',np.round(split-0.1,2))
        for clf in clf_models:
#             vanilla_scores = cross_val_score(clf, x, y, cv=kfold, n_jobs=-1)
            num_models=[5,25,45]
            for n in num_models:
                print("[Bagging {0}]".format(clf.__class__.__name__))
                print('Number of Classifiers: ',n)
                bagging_clf = BaggingClassifier(clf,n_estimators=n, random_state=42).fit(x_train,y_train)
#             bagging_scores = cross_val_score(bagging_clf, x, y, cv=kfold, n_jobs=-1)
                y_pred=bagging_clf.predict(x_test)
                print('Accuracy : ',accuracy_score(y_test, y_pred))
            
#             print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std()))
#             print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std()))
        print()

In [158]:
fit_model(x,y)

Test Size:  0.1
[Bagging DecisionTreeClassifier]
Number of Classifiers:  5
Accuracy :  0.8111111111111111
[Bagging DecisionTreeClassifier]
Number of Classifiers:  25
Accuracy :  0.8111111111111111
[Bagging DecisionTreeClassifier]
Number of Classifiers:  45
Accuracy :  0.8222222222222222
[Bagging KNeighborsClassifier]
Number of Classifiers:  5
Accuracy :  0.7888888888888889
[Bagging KNeighborsClassifier]
Number of Classifiers:  25
Accuracy :  0.8444444444444444
[Bagging KNeighborsClassifier]
Number of Classifiers:  45
Accuracy :  0.8444444444444444
[Bagging SVC]
Number of Classifiers:  5
Accuracy :  0.8222222222222222
[Bagging SVC]
Number of Classifiers:  25
Accuracy :  0.8222222222222222
[Bagging SVC]
Number of Classifiers:  45
Accuracy :  0.8222222222222222

Test Size:  0.2
[Bagging DecisionTreeClassifier]
Number of Classifiers:  5
Accuracy :  0.7988826815642458
[Bagging DecisionTreeClassifier]
Number of Classifiers:  25
Accuracy :  0.7988826815642458
[Bagging DecisionTreeClassifier]
